In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# features = ['SWTP Total Influent Flow', 'SWTP Plant 1 Influent Flow', 'SWTP Plant 2 Influent Flow',
#             'Wilsons Gauge Height (ft)', 'James Gauge Height (ft)', 
#             'Fire 120 Hour Rainfall Aggregate', 'Bingham 120 Hour Rainfall Aggregate', 'Field 120 Hour Rainfall Aggregate', 
#             'Springfield Plateau Aquifer Depth to Water Level (ft)', 'Ozark Aquifer Depth to Water Level (ft)']

# features = ['SWTP Total Influent Flow', 'Fire Rainfall (in)', 'Bingham Rainfall (in)']    # subset 1
# features = ['Springfield Plateau Aquifer Depth to Water Level (ft)', 'Fire Rainfall (in)', 
#             'Bingham Rainfall (in)', 'SWTP Total Influent Flow']      # subset 2
# features = ['Springfield Plateau Aquifer Depth to Water Level (ft)', 'Fire Rainfall (in)', 'Bingham Rainfall (in)', 
#             'SWTP Plant 1 Gravity Flow', 'SWTP Plant 2 Influent Flow', 'SWTP Total Influent Flow']  # subset 3
# features = ['SWTP Plant 1 Gravity Flow', 'SWTP Plant 2 Influent Flow', 'SW_Peak_Flow', 'SWTP Total Influent Flow']
# features = ["Ozark Aquifer Depth to Water Level (ft)", "Fire 168 Hour Rainfall Aggregate", 
#             "English 72 Hour Rainfall Aggregate", "Hour", 'SWTP Total Influent Flow']   # SFS with no influent flow stuff
# features = ['SWTP Total Influent Flow', 'Fire Rainfall (in)', 'Bingham Rainfall (in)', 'Field Rainfall (in)',
#             'Pittman Rainfall (in)', 'Waste Rainfall (in)', 'Ozark Aquifer Depth to Water Level (ft)']    # subset 4
# features = ['SWTP Total Influent Flow', 'Fire Rainfall (in)', 'Bingham Rainfall (in)', 'SWTP Plant 1 Gravity Flow']
# features = ['SWTP Total Influent Flow', 'Fire 120 Hour Rainfall Aggregate', 'Bingham 120 Hour Rainfall Aggregate', 'SWTP Plant 1 Gravity Flow']
features = ['SWTP Total Influent Flow', 'Fire Rainfall (in)', 'Bingham Rainfall (in)', 'SWTP Plant 1 Gravity Flow', "HourlySeaLevelPressure"]


dataset = pd.read_csv("Train and Test Data.csv", usecols=features)
# dataset = pd.read_csv("Imputed Data.csv", usecols=features)
arr = np.array(dataset["SWTP Total Influent Flow"])
# dataset.drop(columns=['SWTP Total Influent Flow'], inplace=True)
# features.remove("SWTP Total Influent Flow")
dataset["Target"] = arr         # adding another influent flow feature so that past values can be used to predict future values
values = dataset.values

# linear transformation of each feature from [min, max] to [0, 1]
scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)

In [3]:
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras_tuner.tuners import BayesianOptimization
import os

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def sliding_window(X, y, n_test, slide):
    split_point = X.shape[0] - n_test + slide
    train_X , train_y = X[:split_point, :] , y[:split_point, :]
    test_X , test_y = X[split_point:, :] , y[split_point:, :]
    return train_X, train_y, test_X, test_y

def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value = 5, max_value = 150, step = 5), 
               activation = 'tanh', return_sequences = True, input_shape = (n_steps_in, len(features))))
    model.add(LSTM(units = hp.Int('units', min_value = 5, max_value = 150, step = 5)))
    model.add(Dense(24))   # for predicting 24 hours -- if desire more, change
    model.add(Activation('linear'))
    model.compile(loss = 'mse', metrics = 'mse', optimizer = keras.optimizers.Adam(
        hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4, 1e-5])))
    return model

def build_model_new(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value = 5, max_value = 150, step = 5), 
               activation = 'tanh', input_shape = (n_steps_in, len(features))))
    model.add(LSTM(units = hp.Int('units', min_value = 5, max_value = 150, step = 5)))
    model.add(Dense(24))   # for predicting 24 hours -- if desire more, change
    model.add(Activation('tanh'))
    model.compile(loss = 'mse', metrics = 'mse', optimizer = keras.optimizers.Adam(
        hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4, 1e-5])))
    return model

def invNormalize(arr, minimum, maximum):
    return (maximum - minimum) * arr + minimum

def predict(model, test_X):
    # to be able to inverse scale predictions
    df = pd.read_csv("Train and Test Data.csv", usecols=["SWTP Total Influent Flow"])
    arr = np.array(df["SWTP Total Influent Flow"])
    maximum = np.max(arr)
    minimum = np.min(arr)

    #predictions and rescaling to [min, max]
    y_pred = model.predict(test_X)
    y_pred_inv = np.array([invNormalize(x, minimum, maximum) for x in y_pred])
    test_y_inv = np.array([invNormalize(x, minimum, maximum) for x in test_y])
    print("y_pred_inv:",y_pred_inv.shape)
    print("test_y_inv:",y_pred_inv.shape)
    
    return y_pred_inv, test_y_inv

def saveResults(path, firstMSE, avgMSE, n_epochs, features, hours=36):
    txt = "n_steps_in = " + str(hours)
    txt += "\nepochs = " + str(n_epochs)
    txt += "\nFirst 96 Avg MSE: " + str(round(firstMSE, 4))
    txt += "\nTotal Avg MSE: " + str(round(avgMSE, 4))
    txt += "\n\nForm:\nLSTM\nLSTM\nDense(24)\nActivation('linear')\n\n"
    txt += str(features)
    with open(path + "\\results.txt", 'w') as f:
        f.write(txt)

def saveResultsSimple(path, firstMSE, avgMSE, n_epochs, features, hours=36):
    txt = "n_steps_in = " + str(hours)
    txt += "\nepochs = " + str(n_epochs)
    txt += "\nFirst 96 Avg MSE: " + str(round(firstMSE, 4))
    txt += "\nTotal Avg MSE: " + str(round(avgMSE, 4))
    txt += "\n\nForm:\nLSTM\nDense(24)\nActivation('linear')\n\n"
    txt += str(features)
    # txt += "\n\nRainfall weight *= 2"
    with open(path + "\\results.txt", 'w') as f:
        f.write(txt)


In [ ]:
n_steps_in = 36
n_steps_out = 24

# covert into input/output
X, y = split_sequences(scaled, n_steps_in, n_steps_out)
print ("X.shape" , X.shape)                             # [rows, time lags backward, features]
print ("y.shape" , y.shape)                             # [rows, future time values]

epochList = [4, 5, 6, 7]
n_tests = [8760, 6552, 4344, 2280]                      
lst = []

start = 3
for i in range(start, len(epochList[start:start+1]) + start):
# for i in range(len(epochList)):
    # path = "feature tuning\\keras_tuner_attempt"
    path = "C:\\Users\\natha\\Desktop\\Undergrad\\Spring2022\\MTH 596 PIC Math\\Project - Group 2\\Project\\Forecasting\\feature tuning\\keras_tuner_attempt"
    path += str(i+1+6)
    modelPath = path + "\\model"
    project_title = "keras_tuner_attempt" + str(i+1+6)
    n_epochs = epochList[i]
    print("Number epochs:", n_epochs)
    for n in n_tests[:1]:               
        for s in range(1):
            train_X, train_y, test_X, test_y = sliding_window(X, y, n, 15 * s)

            # tuning model with keras tuner
            bayesian_opt_tuner = BayesianOptimization(
                build_model,
                # build_model_new,
                objective='mse',
                max_trials=3,
                executions_per_trial=1,
                directory=os.path.normpath('C:/Users/natha/Desktop/Undergrad/Spring2022/MTH 596 PIC Math/Project - Group 2/Project/Forecasting/feature tuning'),
                project_name=project_title,
                overwrite=True)
            bayesian_opt_tuner.search(train_X, train_y, epochs=n_epochs,
                validation_data=(test_X, test_y),
                validation_split=0.2, verbose=1)
            bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)
            model = bayes_opt_model_best_model[0]
            model.save(modelPath)
            # model = keras.models.load_model(modelPath)

            string = "Number epochs = " + str(n_epochs)
            with open(path + "\\note.txt", 'w') as f:
                f.write(string)

            pred_y_inv, test_y_inv = predict(model, test_X)
            firstMSE = 0
            for k in range(96):
                firstMSE += mean_squared_error(pred_y_inv[k], test_y_inv[k])
            # print("First 10 Avg MSE:", firstMSE/10)

            totalAvgMSE = 0
            for i in range(test_y.shape[0]):
                totalAvgMSE += mean_squared_error(test_y_inv[i], pred_y_inv[i])
            totalAvgMSE = totalAvgMSE / test_y.shape[0]
            # saveResultsSimple(path, firstMSE/96, totalAvgMSE, n_epochs, features)
            saveResults(path, firstMSE/96, totalAvgMSE, n_epochs, features)

            lst.append((n_epochs, round(firstMSE/96, 4), round(totalAvgMSE, 4)))
print(lst)

In [ ]:
model = keras.models.load_model("C:\\Users\\natha\\Desktop\\Undergrad\\Spring2022\\MTH 596 PIC Math\\Project - Group 2\\Project\\Forecasting\\feature tuning\\keras_tuner_attempt10\\model")
model.summary()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt

def weightedMSE(arr):
    results = []
    x1 = 0
    A = np.array([[x1, 1-x1]])
    for tup in arr:
        x = np.array([[tup[1]], [tup[2]]])
        b = np.matmul(A, x)
        results.append((tup[0], b[0][0]))
    results.sort(key = lambda x:x[0])
    return results

bestEpochsfeatSubset1 = [(3, 14.4393, 63.0615), (4, 11.7961, 63.7118), (5, 12.4332, 62.5518), (6, 27.4915, 62.4562)]
bestEpochsfeatSubset2 = [(3, 35.0325, 217.5524), (4, 37.7734, 166.1138), (5, 162.7366, 117.081), (6, 77.5023, 155.0186)]
bestEpochsfeatSubset3 = [(3, 34.8093, 86.7812), (4, 24.0673, 66.3212), (5, 37.6574, 74.5868), (6, 39.6214, 80.9921)]
originalSubset = [(3, 201.0433, 108.7818), (4, 93.3019, 67.6975), (5, 27.4284, 74.1717), (6, 24.6906, 66.4358)]
originalResults = weightedMSE(originalSubset)
results1 = weightedMSE(bestEpochsfeatSubset1)
results2 = weightedMSE(bestEpochsfeatSubset2)
results3 = weightedMSE(bestEpochsfeatSubset3)

# graphing
resultList = [originalResults, results1, results2, results3]
subsets = ["original"] + ["subset" + str(i) for i in range(1, len(resultList))]
for results, label in zip(resultList, subsets):
    x = [tup[0] for tup in results]
    y = [tup[1] for tup in results]
    plt.plot(x, y, label=label)
    plt.scatter(x, y)
plt.legend()
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("Weighted MSE Sum")
plt.show()

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
%matplotlib qt

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# features = ['SWTP Total Influent Flow', 'Fire Rainfall (in)', 'Bingham Rainfall (in)']
# features = ['SWTP Total Influent Flow', 'Fire Rainfall (in)', 'Bingham Rainfall (in)', 'SWTP Plant 1 Gravity Flow']
features = ['SWTP Total Influent Flow', 'Fire Rainfall (in)', 'Bingham Rainfall (in)', 'SWTP Plant 1 Gravity Flow', "HourlySeaLevelPressure"]


dataset = pd.read_csv("Imputed Data.csv", usecols=features)
arr = np.array(dataset["SWTP Total Influent Flow"])
dataset["Target"] = arr         # adding another influent flow feature so that past values can be used to predict future values
values = dataset.values

# linear transformation of each feature from [min, max] to [0, 1]
scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)


# n_steps_in = 48
n_steps_in = 36
n_steps_out = 24
X, y = split_sequences(scaled, n_steps_in, n_steps_out)
train_X, train_y, test_X, test_y = sliding_window(X, y, 8760, 0)    # a full year of data

# modelPath = "48 hour epoch tuning\\keras_tuner_attempt14\\model"
# modelPath = "36 hour epoch tuning\\keras_tuner_attempt7\\model"
# modelPath = "feature tuning\\subset 1\\keras_tuner_attempt3\\model" # did v well in long term
# modelPath = "feature tuning\\extraneous\\keras_tuner_attempt8\\model" # avg mse of 58.5759
modelPath = "feature tuning\\subset 5\\keras_tuner_attempt8\\model" # avg mse of 60.5839 with low initial
model = model = keras.models.load_model(modelPath)
y_pred_inv, y_test_inv = predict(model, test_X)

x = list(range(len(y_pred_inv)))
y = [mean_squared_error(y_pred_inv[i], y_test_inv[i]) for i in range(len(y_pred_inv))]



# plotting the mse of 24 hour future predictions
df = pd.read_csv("Imputed Data.csv", usecols=["DateTime", "Fire Rainfall (in)", "Fire 168 Hour Rainfall Aggregate"], parse_dates=["DateTime"])
dates = np.array(df["DateTime"])
dates = dates[-1*test_y.shape[0]-24:-24]

fig, ax = plt.subplots()
formatter = mdates.ConciseDateFormatter(ax.xaxis.get_major_locator(), formats=["%Y", "%Y-%b", "%b-%d", "%d %H:%M", "%d %H:%M", "%H:%M"])
locator = mdates.AutoDateLocator()
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(locator)
fig.autofmt_xdate()

ax.set_title("RMSE of 24 Hour Predictions")
ax.plot(dates, np.sqrt(y))  # root mean square error
plt.show()

OSError: No file or directory found at feature tuning\subset 5\keras_tuner_attempt8\model